## Search Engines with Tools and Agents

In [1]:
#create tool 
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper # this will be used to search for information on the wekipedia and arxiv
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun # this will be used to run the api wrappers above

### Used the Wikipedia inbuilt tool

In [2]:
#Now let's build the wrapper for the tools above
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)

#run the api wrapper 
wikipedia_tool = WikipediaQueryRun(api_wrapper = wikipedia_api_wrapper)

wikipedia_tool.name


'wikipedia'

In [3]:
wikipedia_tool.run("what is langsmith?")

'No good Wikipedia Search Result was found'

### use the Arxiv inbuilt tool

In [4]:
#buit the api wrapper
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results= 1, doc_content_chars_max=250)

#run the api wrapper

arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_api_wrapper)

arxiv_tool.name

'arxiv'

In [5]:
#test the arxiv tool
arxiv_tool.run("what is langsmith?")

'Published: 2018-05-17\nTitle: What is "fundamental"?\nAuthors: Matt Visser\nSummary: Our collective views regarding the question "what is fundamental?" are\ncontinually evolving. These ontological shifts in what we regard as fundamental\nare largely drive'

In [6]:
#combining the two tools
tools = [wikipedia_tool, arxiv_tool]

### Create your own custom tools

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

hf_api_key = os.getenv("HF_API_KEY")

### custom tools [RAG TOOLS]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Loading a web page and put it in vector store 

In [ ]:
#load the webpage 
loader = WebBaseLoader(web_paths = ["https://docs.smith.langchain.com/"])

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

split_documents = text_splitter.split_documents(documents)

vectorstore = FAISS.from_documents(split_documents, HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-V2'))

retriever = vectorstore.as_retriever()

retriever

/home/aljebra/Q&A_Chatbots/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x76902f39fe30>, search_kwargs={})

In [9]:
retriever.invoke("what is langsmith")

[Document(id='b262de20-86ee-4b89-aab8-3d51e1c07dde', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly an

### convert the retriever into a tool (in other to use it as a tool)

In [10]:
from langchain.tools.retriever import create_retriever_tool

In [11]:
retriever_tool = create_retriever_tool(retriever, 'langsmith_retriever', 'Retrieves information from langsmith')

retriever_tool.name

'langsmith_retriever'

In [12]:
retriever_tool.invoke("what is langsmith")

"\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with auto

In [ ]:
#update the list of tools
tools = [ wikipedia_tool, arxiv_tool, retriever_tool]

tools

[Tool(name='langsmith_retriever', description='Retrieves information from langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x76916d6d09a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x76902f39fe30>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x76916d3ab240>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x76902f39fe30>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separat

### Run tools with Agent and LLM models

In [14]:
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")


model = ChatGroq(model='deepseek-r1-distill-llama-70b', api_key=groq_api_key)


model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x769080cf5850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x769080cf6990>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
model.invoke("what is langsmith?")

AIMessage(content='<think>\n\n</think>\n\nAs of my last update in July 2024, there is no widely recognized or established entity, tool, or concept known as "langsmith" in the fields of technology, linguistics, or software development. It\'s possible that "langsmith" could be a niche tool, a new product, or a term specific to a particular community or industry that hasn\'t gained broader recognition yet.\n\nIf "langsmith" is related to language or software development, it might be a tool for:\n\n1. **Language Learning**: A platform or app designed to help users learn languages.\n2. **Programming**: A tool or framework for coding, perhaps related to language modeling or natural language processing.\n3. **Linguistics**: A resource for studying or analyzing languages.\n\nIf you have more context or details about "langsmith," I could provide a more specific answer. Otherwise, it might be helpful to check recent developments or niche communities that might use this term.', additional_kwargs=

### prompt template

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', """You are a helpful ai assistance that can use tool to answer question.
         You have access to:
         - langsmith_retriever : for information about langsmith from its official docs
         - wikipedia: for general encyclopaedia information
         - arxiv : for acedemic paper and research

         when answering questions:
         1. use the most relevant tool for the queries.
         2. if one tool doesn't provide good result try another 
         3. provide a clear and comprehensive answer based on the information received
         4. if you can't find relevant answer, say so clearly."""
         
         ),
        ('human', '{input}'),
        MessagesPlaceholder("agent_scratchpad")
    ]
)

### Now create the agent 

In [31]:
from langchain.agents import create_tool_calling_agent

### To execute the prompt, llm and tools together in the form of chain - use agent

In [32]:
# Create the agent
agent = create_tool_calling_agent(
    llm=model,         # your Groq Llama3 model
    tools=tools,       # your list of tools
    prompt=prompt      # your prompt template
)

In [33]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageCh

### Run the Agent

In [34]:
from langchain.agents import AgentExecutor

In [35]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "what is langsmith"})



> Entering new AgentExecutor chain...



Invoking: `langsmith_retriever` with `{'query': 'what is langsmith'}`







Get started with LangSmith | 🦜️🛠️ LangSmith








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on yo

{'input': 'what is langsmith',
 'output': "LangSmith is a platform designed for building production-grade Large Language Model (LLM) applications. It provides tools and features to help developers monitor, evaluate, and optimize their LLM applications with confidence. Key features include:\n\n1. **Observability**: LangSmith offers LLM-native observability, allowing developers to analyze traces, configure metrics, dashboards, and alerts. This is particularly important for LLMs, which can produce non-deterministic results, making them challenging to debug.\n\n2. **Evals**: This feature enables the creation and management of high-quality evaluation datasets and metrics to test and optimize LLM applications. It includes tools for running evaluations, analyzing results, and collecting human feedback to improve application performance.\n\n3. **Prompt Engineering**: LangSmith provides tools to facilitate prompt engineering, helping developers find the most effective prompts for their applicat

In [37]:
agent_executor.invoke({"input": "what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, asMachine learning is a field of study that gives computers the ability to learn without being explicitly programmed. It involves the use of algorithms that can learn from and make decisions based on data. This is a key component of artificial intelligence and data science. Machine learning can be categorized into several types, including:

1. **Supervised Learning**: The algorithm is trained on labeled data, meaning the data is already tagged with the correct answers. The goal is to learn a mapping from input data to output labels.

2. **Unsupervised Learning**: The algorithm is trained on unlabeled data. The goal is to find hidden patterns or g

{'input': 'what is machine learning',
 'output': 'Machine learning is a field of study that gives computers the ability to learn without being explicitly programmed. It involves the use of algorithms that can learn from and make decisions based on data. This is a key component of artificial intelligence and data science. Machine learning can be categorized into several types, including:\n\n1. **Supervised Learning**: The algorithm is trained on labeled data, meaning the data is already tagged with the correct answers. The goal is to learn a mapping from input data to output labels.\n\n2. **Unsupervised Learning**: The algorithm is trained on unlabeled data. The goal is to find hidden patterns or groupings in the data.\n\n3. **Reinforcement Learning**: The algorithm learns by interacting with an environment and receiving rewards or penalties for its actions. The goal is to learn a policy that maximizes the cumulative reward.\n\nMachine learning is used in a wide range of applications, i

In [38]:
agent_executor.invoke({"input": "what is programming"})



> Entering new AgentExecutor chain...
Programming is the process of designing, writing, testing, and maintaining the source code of computer programs. It involves a variety of activities, including analysis, generating algorithms, and coding. Programming languages are used to write the source code, which is then compiled or interpreted to create the final executable program.

### Key Points:
1. **Definition**: Programming is the systematic process of creating software.
2. **Steps**: It involves problem analysis, algorithm design, coding, testing, and debugging.
3. **Languages**: There are numerous programming languages, each with its own syntax and use cases, such as Python, Java, C++, JavaScript, etc.
4. **Skills**: A programmer needs logical thinking, problem-solving abilities, and attention to detail.

If you'd like more detailed information on a specific aspect of programming, feel free to ask!

> Finished chain.


{'input': 'what is programming',
 'output': "Programming is the process of designing, writing, testing, and maintaining the source code of computer programs. It involves a variety of activities, including analysis, generating algorithms, and coding. Programming languages are used to write the source code, which is then compiled or interpreted to create the final executable program.\n\n### Key Points:\n1. **Definition**: Programming is the systematic process of creating software.\n2. **Steps**: It involves problem analysis, algorithm design, coding, testing, and debugging.\n3. **Languages**: There are numerous programming languages, each with its own syntax and use cases, such as Python, Java, C++, JavaScript, etc.\n4. **Skills**: A programmer needs logical thinking, problem-solving abilities, and attention to detail.\n\nIf you'd like more detailed information on a specific aspect of programming, feel free to ask!"}

In [39]:
agent_executor.invoke({"input": "tell me more about programming"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'programming'}`




/home/aljebra/Q&A_Chatbots/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/aljebra/Q&A_Chatbots/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was found<tool_call>{"

> Finished chain.


{'input': 'tell me more about programming', 'output': '<tool_call>{"'}